# Send orders to Kinesis

In this file we will read the orders from the json files and publish each line as an event to Kinesis.

In [ ]:
import com.amazonaws.services.kinesis.model.PutRecordRequest
import com.amazonaws.services.kinesis.AmazonKinesisClientBuilder
import com.amazonaws.auth.{AWSStaticCredentialsProvider, BasicAWSCredentials}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import collection.JavaConverters._

import java.nio.ByteBuffer
import scala.util.Random

import scala.util.Random
import java.lang.reflect.{Type, ParameterizedType}
import com.fasterxml.jackson.databind.ObjectMapper
import com.fasterxml.jackson.module.scala.DefaultScalaModule
import com.fasterxml.jackson.annotation.JsonProperty;
import com.fasterxml.jackson.core.`type`.TypeReference;

In [ ]:
val awsAccessKeyId = "******"
val awsSecretKey = "*******"
val kinesisStreamName = "Jedha-orders"
val kinesisRegion = "eu-west-1" 

sc.hadoopConfiguration.set("fs.s3a.access.key", awsAccessKeyId)
sc.hadoopConfiguration.set("fs.s3a.secret.key", awsSecretKey)

we read the orders

In [ ]:
var orders = spark.read.text("s3://output-databricks/data/orders-part*.json")

we publich each line to Nakadi

In [ ]:
val kinesisClient = AmazonKinesisClientBuilder.standard()
  .withRegion(kinesisRegion)
  .withCredentials(new AWSStaticCredentialsProvider(new BasicAWSCredentials(awsAccessKeyId, awsSecretKey)))
  .build()

var lastSequenceNumber: String = null

val it = orders.toLocalIterator

while (it.hasNext) {
  val time = System.currentTimeMillis
  
  val data = it.next()(0).asInstanceOf[String] + "\n"
  val request = new PutRecordRequest()
      .withStreamName(kinesisStreamName)
      .withPartitionKey("some-key")
      .withData(ByteBuffer.wrap(data.getBytes()))
  if (lastSequenceNumber != null) {
    request.setSequenceNumberForOrdering(lastSequenceNumber)
  }    
  val result = kinesisClient.putRecord(request)
  lastSequenceNumber = result.getSequenceNumber()
  println(result)
  Thread.sleep(math.max(1000 - (System.currentTimeMillis - time), 0)) // loop around every ~10 seconds 
}